# Credit Risk Ensemble Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Initial imports
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier 






# Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)

df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [5]:
df.isnull().values.any()

False

# Split the Data into Training and Testing

In [6]:
# Create our features
X = df.copy()
X.drop(columns=["loan_status"], axis=1, inplace=True)
X.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,own,52800,0.431818,5,1,22800
1,8400.0,6.692,own,43600,0.311927,3,0,13600
2,9000.0,6.963,rent,46100,0.349241,3,0,16100
3,10700.0,7.664,own,52700,0.430740,5,1,22700
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000


In [7]:
# Use `get_dummies` to generate the features for homeowner
X = pd.get_dummies(X, columns=['homeowner']).copy()
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
0,10700.0,7.672,52800,0.431818,5,1,22800,0,1,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0,1,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0,0,1
3,10700.0,7.664,52700,0.430740,5,1,22700,0,1,0
4,10800.0,7.698,53000,0.433962,5,1,23000,1,0,0


In [8]:
# Create our target
y = df["loan_status"]
y.describe()

#y = df["loan_status"].values.reshape(-1, 1)
#y

count        77536
unique           2
top       low_risk
freq         75036
Name: loan_status, dtype: object

In [9]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [10]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [15]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [16]:
#Use the StandardScaler to scale the features data - X_train and x_training.
# Creating StandardScaler instance
from imblearn.ensemble import EasyEnsembleClassifier 
scaler = StandardScaler()

In [17]:
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)

In [18]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [19]:
# Resample the training data with the BalancedRandomForestClassifier
#from imblearn.ensemble import BalancedRandomForestClassifier
rf_model = BalancedRandomForestClassifier(n_estimators=500, random_state=78)
rf_model.fit(X_train_scaled, y_train)
y_pred = rf_model.predict(X_test_scaled)

In [20]:
# Calculated the balanced accuracy score
acc_score = accuracy_score(y_test, y_pred)
print(f"Balanced Accuracy Score : {acc_score}")

Balanced Accuracy Score : 0.9924164259182832


In [21]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [22]:
# Displaying results
print("Confusion Matrix")
print("----------------------------------")
display(cm_df)


Confusion Matrix
----------------------------------


,Predicted 0,Predicted 1
Actual 0,597,3
Actual 1,144,18640


In [23]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.81      0.99      0.99      0.89      0.99      0.99       600
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18784

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



In [24]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
sorted(zip(importances, X.columns), reverse=True)

[(0.19368205582905115, 'debt_to_income'),
 (0.17640062546533494, 'borrower_income'),
 (0.17542517452307171, 'interest_rate'),
 (0.15873325546977599, 'total_debt'),
 (0.13971022130219177, 'num_of_accounts'),
 (0.134333563589964, 'loan_size'),
 (0.01915480272100321, 'derogatory_marks'),
 (0.001084214428557019, 'homeowner_own'),
 (0.0009456536391961588, 'homeowner_mortgage'),
 (0.0005304330318540283, 'homeowner_rent')]

### Easy Ensemble Classifier

In [25]:
# Train the EasyEnsembleClassifier
#from imblearn.ensemble import EasyEnsembleClassifier 
eec = EasyEnsembleClassifier(random_state=78)
eec.fit(X_train_scaled, y_train) 

EasyEnsembleClassifier(random_state=78)

In [26]:

y_pred = eec.predict(X_test_scaled)
print(confusion_matrix(y_test, y_pred))

[[  597     3]
 [  117 18667]]


In [27]:
# Calculated the balanced accuracy score
acc_score = accuracy_score(y_test, y_pred)
print(f"Balanced Accuracy Score : {acc_score}")

Balanced Accuracy Score : 0.9938093272802311


In [28]:
# Display the confusion matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [29]:
# Displaying results
print("Confusion Matrix")
print("----------------------------------")
display(cm_df)

Confusion Matrix
----------------------------------


,Predicted 0,Predicted 1
Actual 0,597,3
Actual 1,117,18667


In [30]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.99      0.99      0.91      0.99      0.99       600
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18784

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



Use the above to answer the following questions:

Which model had the best balanced accuracy score?
The easy ensemble classifier
Which model had the best recall score?
Both models had the same score of 0.99
Which model had the best geometric mean score?
The geometrical mean was the same at 0.99
What are the top three features?
Debt to income, borrower income and interest rate